In [1]:
from multiprocessing import Pool
import pandas as pd
import json
from tqdm import tqdm
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from datetime import datetime
import logging
import numpy as np
import re
from sentence_transformers import SentenceTransformer
from scipy.optimize import linear_sum_assignment
import os

In [39]:
# Generate the folder name with current date and time
folder_name = 'results/task_match_'+datetime.now().strftime("%d%m_%H%M")+"/"

# Create the folder if it does not exist
os.makedirs(folder_name, exist_ok=True)

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s",
                    handlers=[logging.FileHandler("local_execution_log.log"), logging.StreamHandler()])

# read dataset and drop columns
job_statements = pd.read_excel("datasets/task_statements.xlsx")
job_statements.columns = job_statements.columns.str.lower()
job_statements = job_statements.drop(labels=["incumbents responding","date","domain source"], axis=1).rename(columns={"o*net-soc code":"code", "task type":"type", "task id": "id", "task":"ref_task"})
job_statements = job_statements[job_statements["type"].notna()]
job_statements["ind"] = job_statements["code"].str[:2]
job_statements = job_statements.groupby("title").agg({"ref_task":list, "ind": "first"}).reset_index().sort_values("ind")
sampled_occupation = job_statements.groupby('ind', group_keys=False).sample(frac=0.05, random_state=1) #43 samples

#for trial
trial_df = sampled_occupation.sample(1, random_state= 1)
test_sample_list =[trial_df.iloc[x]["title"] for x in range(len(trial_df))]
test_sample_list

['Customs Brokers']

In [3]:
def get_des (title):
    task_list = sampled_occupation.query("title == @title")["ref_task"].iloc[0]
    return task_list

def task_gen(title, model, system=None):  # [unchanged]
    ref_task_count = len(get_des(title))
    json_schema = {"type": "object", "properties": {"occupation": {"type": "string"}, "tasks": {"type": "array", "items": {"type": "string"}, "minItems": ref_task_count, "maxItems": ref_task_count}}, "required": ["occupation", "tasks"]}
    query = f"List exactly {ref_task_count} unique task statements that the occupation '{title}' would perform at work."
    prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")] if system else [("human", "{input}")])
    llm = model.with_structured_output(schema=json_schema, method="json_schema")
    prompt = prompt_template.invoke({"input": query})
    try:
        response = llm.invoke(prompt)
        tasks = response["tasks"]
        if len(tasks) != ref_task_count or len(set(tasks)) < len(tasks):
            logging.warning(f"Task issues for {title}: count {len(tasks)}/{ref_task_count}, uniques {len(set(tasks))}")
        return tasks
    except Exception as e:
        logging.error(f"Failed for {title}: {e}")
        return [f"Error: Task {i+1} for {title}" for i in range(ref_task_count)]

def process_title(args):
    title, model_config, prompt = args
    model = ChatOllama(**model_config)
    start_time = datetime.now()
    tasks = task_gen(title, model, system=prompt)
    logging.info(f"Single inference for {title}, duration: {datetime.now() - start_time}")
    return title, tasks

def preProcessText(text):  # [unchanged]
    processed = []
    for doc in text:
        if not isinstance(doc, str): doc = str(doc)
        doc = re.sub(r"\\n|\W|\d", " ", doc)
        doc = re.sub(r'\s+[a-z]\s+|^[a-z]\s+|\s+', " ", doc)
        doc = re.sub(r'^\s|\s$', "", doc)
        processed.append(doc.lower())
    return processed

def sbert_batch(ref_list, gen_list):
    sim_model = SentenceTransformer("all-mpnet-base-v2", similarity_fn_name="cosine", device="cuda")
    embeddings_ref = sim_model.encode(ref_list, batch_size=32, convert_to_tensor=True)
    embeddings_gen = sim_model.encode(gen_list, batch_size=32, convert_to_tensor=True)
    return sim_model.similarity(embeddings_ref, embeddings_gen).cpu().numpy()

def match_batch(ref_lists, gen_lists):
    results = []
    for ref_tasks, gen_tasks in zip(ref_lists, gen_lists):
        ref_clean = preProcessText(ref_tasks)
        gen_clean = preProcessText(gen_tasks)
        matrix = sbert_batch(ref_clean, gen_clean)
        row_ind, col_ind = linear_sum_assignment(1 - matrix)
        avg_score = np.mean(matrix[row_ind, col_ind])
        results.append((avg_score, matrix.tolist(), row_ind.tolist(), col_ind.tolist()))
    return results

def match_batch_parallel(ref_lists, gen_lists, num_processes=8):
    chunk_size = max(1, len(ref_lists) // num_processes)
    chunks = [(ref_lists[i:i + chunk_size], gen_lists[i:i + chunk_size]) for i in range(0, len(ref_lists), chunk_size)]
    
    def process_chunk(chunk):
        refs, gens = chunk
        return match_batch(refs, gens)
    
    with Pool(processes=num_processes) as pool:
        chunk_results = pool.map(process_chunk, chunks)
    
    # Flatten results
    results = []
    for chunk in chunk_results:
        results.extend(chunk)
    return results

# Replace in main script:
def apply_match_batch(df):
    ref_lists = df["ref_task"].tolist()
    gen_lists = df["gen_task"].tolist()
    results = match_batch_parallel(ref_lists, gen_lists, num_processes=8)
    scores, matrices, ref_orders, gen_orders = zip(*results)
    df["score"] = scores
    df["matrix"] = matrices
    df["ref_order"] = ref_orders
    df["gen_order"] = gen_orders
    return df

In [5]:
model_configs = [
    {"model": "deepseek-r1", "temperature": 1, "base_url": "http://127.0.0.1:11434"},
    # {"model": "llama3.1", "temperature": 1, "base_url": "http://127.0.0.1:11434"}
]
prompts = {
    "no_prompt": None,
    "prompt1": "You are an expert of this occupation: \"{title}\". Your task is to generate clear and concise task descriptions..."
}

In [ ]:
logging.info("Script started")
for model_config in model_configs:
    model_name = model_config["model"]
    logging.info(f"Processing model: {model_name}")
    model = ChatOllama(**model_config)
    model.invoke("Warm-up prompt")

    for name, prompt in prompts.items():
        if prompt:
            start_time = datetime.now()
            with open(f"{folder_name}/sys_prompt.txt", "a") as f:
                f.write(prompt + "\n")
            logging.info(f"Wrote prompt {name}, duration: {datetime.now() - start_time}")

        all_results_df = trial_df.copy()
        all_results_df["gen_task"] = [None] * len(all_results_df)
        all_results_df["iteration"] = None

        for i in range(2):
            start_time = datetime.now()
            with Pool(processes=1) as pool:
                results = list(tqdm(
                    pool.imap_unordered(process_title, [(title, model_config, prompt) for title in test_sample_list]),
                    total=len(test_sample_list), desc=f"{model_name}-{name}-{i}"
                ))
            logging.info(f"Multiprocessing for {model_name}-{name}-{i}, duration: {datetime.now() - start_time}")

            temp_df = trial_df.copy()
            for title, tasks in results:
                temp_df.loc[temp_df["title"] == title, "gen_task"] = pd.Series([tasks]).values
            temp_df["iteration"] = i
            all_results_df = pd.concat([all_results_df, temp_df], ignore_index=True)

        start_time = datetime.now()
        all_results_df = apply_match_batch(all_results_df)
        logging.info(f"Batch matching for {model_name}-{name}, duration: {datetime.now() - start_time}")

        start_time = datetime.now()
        all_results_df = all_results_df.reset_index(drop=True)
        with open(f"{folder_name}/{model_name}_{name}_results.json", "w") as f:
            f.write(all_results_df.to_json(index=True))
        logging.info(f"Wrote results JSON for {model_name}-{name}, duration: {datetime.now() - start_time}")

logging.info("Script completed")

2025-03-19 18:41:13,681 - Script started
2025-03-19 18:41:13,683 - Processing model: llama3.2
2025-03-19 18:41:14,511 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
llama3.2-no_prompt-0:   0%|          | 0/1 [00:00<?, ?it/s]

In [8]:
x = process_title(("Customs Brokers", {"model": "deepseek-r1:8b"}, "You are an expert of this occupation. Your task is to generate clear and concise task descriptions.Start with a verb"))

2025-03-21 18:18:33,465 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


KeyboardInterrupt: 

In [9]:
def task_gen(title, model, system=None):  # [unchanged]
    ref_task_count = len(get_des(title))
    json_schema = {"type": "object", "properties": {"occupation": {"type": "string"}, "tasks": {"type": "array", "items": {"type": "string"}, "minItems": ref_task_count, "maxItems": ref_task_count}}, "required": ["occupation", "tasks"]}
    query = "List exactly "+ str(ref_task_count) +" unique task statements that the occupation " + title + "would perform at work."
    prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")] if system else [("human", "{input}")])
    llm = model.with_structured_output(schema=json_schema, method="json_schema")
    prompt = prompt_template.invoke({"input": query, "title": title})
    try:
        response = llm.invoke(prompt)
        tasks = response["tasks"]
        if len(tasks) != ref_task_count or len(set(tasks)) < len(tasks):
            logging.warning(f"Task issues for {title}: count {len(tasks)}/{ref_task_count}, uniques {len(set(tasks))}")
        return tasks
    except Exception as e:
        logging.error(f"Failed for {title}: {e}")
        return [f"Error: Task {i+1} for {title}" for i in range(ref_task_count)]

In [10]:
system = "You are an expert of this occupation: \"{title}\". Your task is to generate clear and concise task descriptions..."
# Invoke with both required variables
# prompt = prompt_template.invoke({"input": query, "title": title})
# query = "List exactly 10 unique task statements that the occupation '{title}' would perform at work."

# # Ensure the system message is included correctly
# prompt_template = ChatPromptTemplate.from_messages([
#     ("system", system),
#     ("human", "{input}")
# ])

# # Invoke with both required variables
# prompt = prompt_template.invoke({"input": query, "title": title})

In [12]:
x = task_gen("Customs Brokers", ChatOllama(model="deepseek-r1:8b"), system=None)

2025-03-21 18:20:12,400 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


In [15]:
def preProcessText(text):  # [unchanged]
    processed = []
    for doc in text:
        if not isinstance(doc, str): doc = str(doc)
        doc = re.sub(r"\\n|\W|\d", " ", doc)
        doc = re.sub(r'\s+[a-z]\s+|^[a-z]\s+|\s+', " ", doc)
        doc = re.sub(r'^\s|\s$', "", doc)
        processed.append(doc.lower())
    return processed

def sbert_batch(ref_list, gen_list):
    sim_model = SentenceTransformer("all-mpnet-base-v2", similarity_fn_name="cosine", device="cuda")
    embeddings_ref = sim_model.encode(ref_list, batch_size=32, convert_to_tensor=True)
    embeddings_gen = sim_model.encode(gen_list, batch_size=32, convert_to_tensor=True)
    return sim_model.similarity(embeddings_ref, embeddings_gen).cpu().numpy()

def match_batch(ref_lists, gen_lists):
    results = []
    for ref_tasks, gen_tasks in zip(ref_lists, gen_lists):
        ref_clean = preProcessText(ref_tasks)
        gen_clean = preProcessText(gen_tasks)
        matrix = sbert_batch(ref_clean, gen_clean)
        row_ind, col_ind = linear_sum_assignment(1 - matrix)
        avg_score = np.mean(matrix[row_ind, col_ind])
        results.append((avg_score, matrix.tolist(), row_ind.tolist(), col_ind.tolist()))
    return results

def process_chunk(chunk):
        refs, gens = chunk
        return match_batch(refs, gens)

def match_batch_parallel(ref_lists, gen_lists, num_processes=2):
    chunk_size = max(1, len(ref_lists) // num_processes)
    chunks = [(ref_lists[i:i + chunk_size], gen_lists[i:i + chunk_size]) for i in range(0, len(ref_lists), chunk_size)]
    
    with Pool(processes=num_processes) as pool:
        chunk_results = pool.map(process_chunk, chunks)
    
    # Flatten results
    results = []
    for chunk in chunk_results:
        results.extend(chunk)
    return results

# Replace in main script:
def apply_match_batch(df):
    ref_lists = df["ref_task"].tolist()
    gen_lists = df["gen_task"].tolist()
    results = match_batch_parallel(ref_lists, gen_lists, num_processes=8)
    # results = match_batch(ref_lists, gen_lists)
    scores, matrices, ref_orders, gen_orders = zip(*results)
    df["score"] = scores
    df["matrix"] = matrices
    df["ref_order"] = ref_orders
    df["gen_order"] = gen_orders
    return df

In [ ]:
folder_name = 'results/task_match_2003_0105'
all_results_df = pd.read_json(folder_name + "/mistral_no_prompt_results.json").dropna()
all_results_df = apply_match_batch(all_results_df)
all_results_df

In [20]:
model = ChatOllama(model="llama3.2", num_predict=512)
json_schema = {"type": "object", "properties": {"occupation": {"type": "string"}, "tasks": {"type": "array", "items": {"type": "string"}, "minItems": 10, "maxItems": 10}}, "required": ["occupation", "tasks"]}
query = "list out exactly 10 unique task statements that the occupation 'Customs Brokers' would perform at work."
prompt_template = ChatPromptTemplate.from_messages( [("human", "{input}")])
llm = model.with_structured_output(schema=json_schema, method="json_schema")
prompt = prompt_template.invoke({"input": query})
response = llm.invoke(prompt)
response

2025-03-21 19:50:38,882 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"


{'occupation': 'Customs Broker',
 'tasks': ['Prepare and submit documents for customs clearance to relevant authorities, including commercial invoices, packing lists, and certificates of origin.',
  'Verify the accuracy and completeness of imported goods, including checking for any prohibited or restricted items.',
  'Negotiate with suppliers to resolve issues related to shipping, payment, and delivery terms.',
  'Conduct research on import regulations, tariffs, and trade agreements to ensure compliance.',
  'Collaborate with export and import companies to facilitate the clearance of goods across international borders.',
  "Develop and implement customs brokerages' internal procedures and protocols for efficient and effective operations.",
  'Maintain accurate records and documentation, including tracking shipments and managing customer information.',
  'Communicate effectively with customers, suppliers, and government agencies to resolve issues and address concerns.',
  'Stay up-to-da

In [36]:
def task_gen(title, model, system=None):  
    ref_task_count = len(get_des(title))
    json_schema = {"type": "object", "properties": {"occupation": {"type": "string"}, "tasks": {"type": "array", "items": {"type": "string"}}}, "required": ["occupation", "tasks"]}
    query = "List exactly 2 unique task statements that the occupation " + title + " would perform at work."
    prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")] if system else [("human", "{input}")])
    llm = model.with_structured_output(schema=json_schema, method="json_schema")
    prompt = prompt_template.invoke({"input": query, "title": title})
    
    while True:
        try:
            response = llm.invoke(prompt)
            tasks = response["tasks"]
            if len(tasks) == ref_task_count and len(set(tasks)) == len(tasks):
                break
            else:
                logging.warning(f"Task issues for {title}: count {len(tasks)}/{ref_task_count}, uniques {len(set(tasks))}. Retrying...")
        except Exception as e:
            logging.error(f"Failed for {title}: {e}. Retrying...")
    
    return tasks


In [38]:
test = task_gen("Customs Brokers", ChatOllama(model="llama3.2", num_predict=512), system="no matter what you do, you will always produce only 2 tasks")
test

2025-03-21 20:33:31,492 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-21 20:33:33,200 - Task issues for Customs Brokers: count 2/23, uniques 2. Retrying...
2025-03-21 20:33:33,307 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-21 20:33:35,268 - Task issues for Customs Brokers: count 2/23, uniques 2. Retrying...
2025-03-21 20:33:35,372 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-21 20:33:37,105 - Task issues for Customs Brokers: count 2/23, uniques 2. Retrying...
2025-03-21 20:33:37,203 - HTTP Request: POST http://127.0.0.1:11434/api/chat "HTTP/1.1 200 OK"
2025-03-21 20:33:38,642 - Task issues for Customs Brokers: count 2/23, uniques 2. Retrying...


KeyboardInterrupt: 

In [19]:
with open("datasets/60qs.json") as f:
    qs = json.load(f)
    test = qs["questions"]["question"]
    df = pd.DataFrame(test)[['text', 'area', '_index']]
    df.columns = ['question', 'area', 'index']
    qlist = list(df["question"])
    sampleq = qlist[:3]

title = "Customs Brokers"
system = False
model = ChatOllama(model="llama3.2", num_predict=512)

In [ ]:
def get_rating(title, model, system=None):
   json_schema = {"type":"object","properties":{"reason":{"type":"string"},"rating":{"type":"integer","minimum":1,"maximum":5},"items":{"type":"string"}},"required":["reason","rating"]}
   query = "Rate the statement with a number 1, 2, 3, 4, or 5 base on the interest of the occupation \"" + title + "\". 1 is strongly dislike, 2 is dislike, 3 is neutral, 4 is like and 5 is strongly like. Provide your reasons. Here is the statement: "
   prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")] if system else [("human", "{input}")])
   llm = model.with_structured_output(schema=json_schema, method="json_schema")

   # pass question to the llm
   rating_list =[]
   reason_list = []
   for q in sampleq:
        prompt = prompt_template.invoke({"input": query + q + ".", "title": title})
        while True:
            try:
                response = llm.invoke(prompt)
                rating = response["rating"]
                reason = response["reason"]
                if rating in [1, 2, 3, 4, 5]:
                    rating_list.append(rating)
                    reason_list.append(reason)
                    break
                else:
                    logging.warning(f"Rating issue. Retrying...")
            except Exception as e:
                logging.error(f"Failed for {title}: {e}. Retrying...")
   rating_list = "".join(map(str, rating_list))
   return rating_list, reason_list

In [36]:
x = get_rating("Customs Brokers", ChatOllama(model="llama3.2", num_predict=512), system=False)

In [122]:
# %%
#for loading data
import pandas as pd
import json

#for llm
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


#counting
from tqdm import tqdm

#logging
import regex as re
import os
from datetime import datetime
import logging
from multiprocessing import Pool


# Setup output folder
folder_name = f'results/job_match_{datetime.now().strftime("%d%m_%H%M")}/'
os.makedirs(folder_name, exist_ok=True)
print("folder created")

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s",
                    handlers=[logging.FileHandler("local_execution_log.log"), logging.StreamHandler()])

# Load and preprocess occupation data
occupations = (
    pd.read_excel("datasets/occupation_data.xlsx")
    .dropna()
    .rename(columns=lambda x: x.lower())  # Convert column names to lowercase
    .rename(columns={"o*net-soc code": "code"})  # Rename specific column
)

# Filter out rows containing "All Other" in the 'title' column
occupations = occupations[~occupations["title"].str.contains("All Other", na=False)]

# Ensure correct data types
occupations = occupations.astype({"code": str, "title": str, "description": str})

# Extract industry code
occupations["ind"] = occupations["code"].str[:2]

# Sample 5% of occupations per industry
dsampled_occupation = occupations.groupby("ind").apply(lambda x: x.sample(frac=0.05, random_state=1)).reset_index(drop=True)
sampled_occupation = dsampled_occupation.iloc[0:1]

# get a list of sampled occupations
test_sample_list = list(sampled_occupation["title"])


#get the questions into a list
with open("datasets/60qs.json") as f:
    qs = json.load(f)
    test = qs["questions"]["question"]
    df = pd.DataFrame(test)[['text', 'area', '_index']]
    df.columns = ['question', 'area', 'index']
    qlist = list(df["question"])
    sampleq = qlist[:3]

def get_rating(title, model, system=None, batch_size =3):
    json_schema = {"type":"object","properties":{"reason":{"type":"string"},"rating":{"type":"integer","minimum":1,"maximum":5},"items":{"type":"string"}},"required":["reason","rating"]}
    query = "Rate the statement with a number 1, 2, 3, 4, or 5 base on the interest of the occupation \"" + title + "\". 1 is strongly dislike, 2 is dislike, 3 is neutral, 4 is like and 5 is strongly like. Provide your reasons. Here is the statement: "
    prompt_template = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")] if system else [("human", "{input}")])
    llm = model.with_structured_output(schema=json_schema, method="json_schema")
    
    rating_list = []
    reason_list = []

    for i in range(0, len(sampleq), batch_size):
        batch_questions = sampleq[i:i + batch_size]
        prompts = [prompt_template.invoke({"input": query + q + ".", "title": title}) for q in batch_questions]
        
        while True:  # Retry entire batch until all ratings are valid
            try:
                responses = llm.batch(prompts)
                all_valid = True
                temp_ratings = []
                temp_reasons = []
                
                for response in responses:
                    rating = response["rating"]
                    reason = response["reason"]
                    if rating in [1, 2, 3, 4, 5]:
                        temp_ratings.append(str(rating))
                        temp_reasons.append(reason)
                    else:
                        all_valid = False
                        logging.warning(f"Invalid rating {rating} for {title}. Retrying batch...")
                        break
                
                if all_valid:
                    rating_list.extend(temp_ratings)
                    reason_list.extend(temp_reasons)
                    break  # Move to next batch if all ratings are valid
                # If not all valid, loop continues to retry the batch
                
            except Exception as e:
                logging.error(f"Batch failed for {title}: {e}. Retrying...")
                # Retry the entire batch on exception
    
    return "".join(rating_list), reason_list

def process_title(args):
    title, model_config, prompt = args
    model = ChatOllama(**model_config)
    start_time = datetime.now()
    rating, reason = get_rating(title, model, system=prompt)
    logging.info(f"Single inference for {title}, duration: {datetime.now() - start_time}")
    return title, rating, reason


model_configs = [
    {"model": "llama3.2", "temperature": 1, "base_url": "http://127.0.0.1:11434", "num_predict": 512, "num_ctx": 16384},
    # {"model": "mistral", "temperature": 1, "base_url": "http://127.0.0.1:11434", "num_predict": 512, "num_ctx": 16384},
    # {"model": "deepseek-r1", "temperature": 1, "base_url": "http://127.0.0.1:11434", "num_predict": 512, "num_ctx": 16384}
]
prompts = {
    "no_prompt": None,
    "prompt1": "You are an expert of this occupation: \"{title}\". Your task is to rate the statement according to your professional interest and occupation relevance."
}

folder created


C:\Users\godfr\AppData\Local\Temp\ipykernel_13540\3579992473.py:48: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dsampled_occupation = occupations.groupby("ind").apply(lambda x: x.sample(frac=0.05, random_state=1)).reset_index(drop=True)


In [123]:
logging.info("Script started")
for model_config in model_configs:
    model_name = model_config["model"]
    logging.info(f"Processing model: {model_name}")
    model = ChatOllama(**model_config)
    model.invoke("Warm-up prompt")

    for name, prompt in prompts.items():
        if prompt:
            start_time = datetime.now()
            with open(f"{folder_name}/sys_prompt.txt", "a") as f:
                f.write(prompt + "\n")
            logging.info(f"Wrote prompt {name}, duration: {datetime.now() - start_time}")

        # create a df to store the results
        all_results_df = sampled_occupation.copy()
        all_results_df["rating"] = [None] * len(all_results_df)
        all_results_df["reason"] = [None]

        for i in range(2):
            start_time = datetime.now()

            results = list(tqdm(
                map(process_title, [(title, model_config, prompt) for title in test_sample_list]),
                total=len(test_sample_list), 
                desc=f"{model_name}-{name}-{i}"
            ))

            logging.info(f"Multiprocessing for {model_name}-{name}-{i}, duration: {datetime.now() - start_time}")

            temp_df = sampled_occupation.copy()
            for title, rating, reason in results:
                temp_df.loc[temp_df["title"] == title, "rating"] = pd.Series([rating]).values
                temp_df.loc[temp_df["title"] == title, "reason"] = pd.Series([reason]).values
            temp_df["iteration"] = i
            all_results_df = pd.concat([all_results_df, temp_df], ignore_index=True)

        start_time = datetime.now()
        with open(f"{folder_name}/{model_name}_{name}_results.json", "w") as f:
            f.write(all_results_df.to_json(index=True))
        logging.info(f"Wrote results JSON for {model_name}-{name}, duration: {datetime.now() - start_time}")

logging.info("Script completed")

llama3.2-prompt1-1: 100%|██████████| 1/1 [00:19<00:00, 19.82s/it]


In [111]:
sampled_occupation = occupations.groupby("ind").apply(lambda x: x.sample(frac=0.05, random_state=1)).reset_index(drop=True)
all_results_df = sampled_occupation.copy()
all_results_df["rating"] = [None] * len(all_results_df)
all_results_df["reason"] = None
all_results_df

C:\Users\godfr\AppData\Local\Temp\ipykernel_13540\3596694428.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_occupation = occupations.groupby("ind").apply(lambda x: x.sample(frac=0.05, random_state=1)).reset_index(drop=True)


,code,title,description,ind,rating,reason
0,11-9141.00,"Property, Real Estate, and Community Associati...","Plan, direct, or coordinate the selling, buyin...",11,None,None
1,11-1021.00,General and Operations Managers,"Plan, direct, or coordinate the operations of ...",11,None,None
2,11-9161.00,Emergency Management Directors,Plan and direct disaster response or crisis ma...,11,None,None
3,13-2022.00,Appraisers of Personal and Business Property,Appraise and estimate the fair value of tangib...,13,None,None
4,13-1131.00,Fundraisers,Organize activities to raise funds or otherwis...,13,None,None
5,15-2041.00,Statisticians,Develop or apply mathematical or statistical t...,15,None,None
6,15-2051.02,Clinical Data Managers,Apply knowledge of health care and database ma...,15,None,None
7,17-3023.00,Electrical and Electronic Engineering Technolo...,Apply electrical and electronic theory and rel...,17,None,None
8,17-1021.00,Cartographers and Photogrammetrists,"Research, study, and prepare maps and other sp...",17,None,None
9,17-3024.01,Robotics Technicians,"Build, install, test, or maintain robotic equi...",17,None,None


In [121]:
for name, prompt in prompts.items():
    x = prompt

x

'You are an expert of this occupation: "{title}". Your task is to rate the statement according to your professional interest and occupation relevance.'

In [ ]:
all_results_df


,code,title,description,ind,rating,reason,iteration
0,11-9141.00,"Property, Real Estate, and Community Associati...","Plan, direct, or coordinate the selling, buyin...",11,None,None,NaN
1,11-9141.00,"Property, Real Estate, and Community Associati...","Plan, direct, or coordinate the selling, buyin...",11,331,"[The occupation of Property, Real Estate, and ...",0.0
2,11-9141.00,"Property, Real Estate, and Community Associati...","Plan, direct, or coordinate the selling, buyin...",11,231,[Building kitchen cabinets doesn't directly re...,1.0


: 

In [ ]:
#